In [1]:
import sys, os, inspect
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from dataset.Dataset import Dataset
from sklearn.metrics import accuracy_score
import torch
from kan.KAN import KAN
from kan.DropKAN import DropKAN
from kan.DropKANLayer import DropKANLayer
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd 

def set_training_mode(module, mode):
    """Set the training mode for a module and all its sub-modules."""
    module.training = mode
    for submodule in module.children():
        set_training_mode(submodule, mode)

In [2]:
work_dir = 'uci datasets/Shuffled/'
log_folder =  'log\\'

header=None
file_name, sep, label = 'german.data-numeric', ',', -1
#file_name, sep, label = 'abalone.data', ',', -1
file_name, sep, label = 'dermatology.data', ',', -1
#file_name, sep, label = 'car.data', ',', -1
#file_name, sep, label = 'adult.data', ',', -1
#file_name, sep, label = 'census-income.data', ',', -1
file_name, sep, label = 'diabetic_data.csv', ',', -1
#file_name, sep, label, header = 'bank-full.csv', ',', -1, 0
file_name, sep, label = 'connect-4.data', ',', -1
#file_name, sep, label = 'semeion.data', ',', -1

 




data = Dataset(work_dir+file_name, sep, label, divide_dataset=False, header=header)

In [3]:
classifier = RandomForestClassifier(n_estimators=111, random_state=0, n_jobs=-1)
data.divide_dataset(classifier,
                                      normalize=True, 
                                      shuffle=False, 
                                      all_features=True, 
                                      all_instances=True, 
                                      evaluate=True, 
                                      partial_sample=False,
                                      metric=accuracy_score)

In [4]:
data.get_validation_accuracy()

0.8060099178447191

In [5]:
data.get_test_accuracy()

0.8078744819419775

In [6]:
dataset = {}
dataset['train_input'] = torch.from_numpy(data.X_train).float()
dataset['test_input'] = torch.from_numpy(data.X_test).float()
dataset['val_input'] = torch.from_numpy(data.X_val).float()
dataset['train_label'] = torch.from_numpy(data.y_train[:, None])
dataset['test_label'] = torch.from_numpy(data.y_test[:, None])
dataset['val_label'] = torch.from_numpy(data.y_val[:, None])

In [117]:

# Log file name
log_file =  log_folder+file_name+'_log.xlsx'

# DataFrame to store logs
log_df = pd.DataFrame(columns=['drop_rate1', 'drop_rate2', 'drop_mode', 'scale', 'validation_accuracy', 'test_accuracy'])

for j in range(5):
    model = DropKAN(seed=j,width=[data.X_train.shape[1], 10,1], grid=3, k=3, bias_trainable=True, sp_trainable=True, sb_trainable=True,
                            symbolic_enabled=False, drop_rate=[None, None], drop_mode='kan', drop_scale=None)
    results = model.train(dataset, opt="Adam", steps=2000, batch=32, lr=0.01, update_grid=False)
    set_training_mode(model, False)
    
    val_acc = torch.mean((torch.round(model(dataset['val_input'])[:,0]) == dataset['val_label'][:,0]).float()).item()
    test_acc = torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float()).item()
    print(0.0, 0.0, 'kan', None, val_acc, test_acc)
    log_df.loc[len(log_df)] = [None, None, 'kan', None, val_acc, test_acc]
    log_df.to_excel(log_file, index=False)


best = 0
for _ in range(10):  # Number of random configurations to test
    random.seed()
    drop_rate1 = random.uniform(0, .5)
    drop_rate2 = random.uniform(0, .5)

    for drop_mode in ['postspline', 'postact', 'dropout']:

        for scale in  [True, False]:
            print(drop_rate1, drop_rate2, drop_mode)
            
            
            
            model = DropKAN(width=[data.X_train.shape[1], 10,1], grid=3, k=3, bias_trainable=True, sp_trainable=True, sb_trainable=True,
                            symbolic_enabled=False, drop_rate=[drop_rate1, drop_rate2], drop_mode=drop_mode, drop_scale=scale)

            results = model.train(dataset, opt="Adam", steps=2000, log=100, batch=32, lr=0.01, update_grid=False)
            set_training_mode(model, False)
            
            val_acc = torch.mean((torch.round(model(dataset['val_input'])[:,0]) == dataset['val_label'][:,0]).float())
            test_acc = torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float())
            
            
                
            new_row = [drop_rate1,drop_rate2,drop_mode,scale,val_acc.item(),test_acc.item()]
            
            log_df.loc[len(log_df)] = new_row
        
            # Save the DataFrame to an Excel file
            log_df.to_excel(log_file, index=False)
            print(val_acc, test_acc)
            print(len(log_df))
            print()

print(f"Log saved to {log_file}")

description: 100%|█████████████████████████████████████████████| 2000/2000 [00:19<00:00, 100.62it/s]


0.0 0.0 kan None 0.9513594508171082 0.9510337114334106


description: 100%|█████████████████████████████████████████████| 2000/2000 [00:19<00:00, 101.03it/s]


0.0 0.0 kan None 0.9494799971580505 0.9501315355300903


description: 100%|█████████████████████████████████████████████| 2000/2000 [00:19<00:00, 101.78it/s]


...

tensor(0.9302) tensor(0.9329)
153

0.31195095235420656 0.44835727332583897 dropout


description: 100%|██████████████████████████████████████████████| 2000/2000 [00:20<00:00, 99.73it/s]


tensor(0.9371) tensor(0.9370)
154

0.31195095235420656 0.44835727332583897 dropout


description: 100%|█████████████████████████████████████████████| 2000/2000 [00:19<00:00, 100.59it/s]


tensor(0.9448) tensor(0.9454)
155

Log saved to log\census-income.data_log.xlsx


In [7]:
log_df = pd.read_excel(log_folder+file_name+'_log.xlsx')


In [15]:
grouped = log_df.groupby(['drop_mode', 'scale'])
best_settings = grouped.apply(lambda x: x.loc[x['validation_accuracy'].idxmax()])

results_df = pd.DataFrame(columns=['dropmode', 'dropscale', 'validation_accuracy', 'test_accuracy'])

for i, (index, setting) in enumerate(best_settings.iterrows()):
    for j in range(1):
        model = DropKAN(
        seed=j,
        width=[dataset['train_input'].shape[1], 10, 1],
        grid=5,
        symbolic_enabled=False,
        drop_rate=[setting['drop_rate1'], setting['drop_rate2']],
        drop_mode=setting['drop_mode'],
        drop_scale=setting['scale'],
        neuron_fun=['mean', 'mean']
        )
        results = model.train(dataset, opt="Adam", steps=300, batch=32, lr=0.01, update_grid=False)
        set_training_mode(model, False)
        
        val_acc = torch.mean((torch.round(model(dataset['val_input'])[:,0]) == dataset['val_label'][:,0]).float()).item()
        test_acc = torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float()).item()
        print(setting['drop_mode'], setting['scale'], val_acc, test_acc)
        results_df.loc[len(results_df)] = [setting['drop_mode'], setting['scale'], val_acc, test_acc]
    

description: 100%|███████████████████████████████████████████████| 300/300 [00:02<00:00, 114.56it/s]


dropout 0.0 0.6586484909057617 0.674141526222229


description: 100%|███████████████████████████████████████████████| 300/300 [00:02<00:00, 113.98it/s]


dropout 1.0 0.6732292175292969 0.682652473449707


description: 100%|███████████████████████████████████████████████| 300/300 [00:02<00:00, 113.68it/s]


...

postact 1.0 0.6698246002197266 0.675399661064148


description: 100%|███████████████████████████████████████████████| 300/300 [00:02<00:00, 113.76it/s]


postspline 0.0 0.6565760970115662 0.6674067378044128


description: 100%|███████████████████████████████████████████████| 300/300 [00:02<00:00, 112.74it/s]


postspline 1.0 0.6743394136428833 0.683910608291626


In [12]:
for j in range(1):
        model = KAN(
        seed=j,
        width=[dataset['train_input'].shape[1], 10, 1],
        grid=3,
            k=3,
        symbolic_enabled=False
        )
        results = model.train(dataset, opt="Adam", steps=2000, batch=32, lr=0.01, update_grid=False)
        set_training_mode(model, False)
        
        val_acc = torch.mean((torch.round(model(dataset['val_input'])[:,0]) == dataset['val_label'][:,0]).float()).item()
        test_acc = torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float()).item()
        print('kan', None, val_acc, test_acc)
        results_df.loc[len(results_df)] = ['kan', -1, val_acc, test_acc]

description:   1%|▌                                               | 21/2000 [00:03<05:20,  6.17it/s]


KeyboardInterrupt: 

In [ ]:
results_df.to_excel(log_folder+file_name+'_results.xlsx', index=False) 

In [68]:
from scipy import stats
from scipy.stats import ranksums


# Create a DataFrame to store significance results
unique_combinations = results_df[['dropmode', 'dropscale']].drop_duplicates().values

significance_results = pd.DataFrame(columns=['comb1', 'comb2', 'p_value', 'significant', 'accuracies1', 'accuracies2', 'std1', 'std2'])


# Compare each pair of combinations
for i in range(len(unique_combinations)):
    for j in range(i + 1, len(unique_combinations)):
        comb1 = unique_combinations[i]
        comb2 = unique_combinations[j]


        if True:
        
            # Get the test accuracies for the two combinations
            accuracies1 = results_df[(results_df['dropmode'] == comb1[0]) &
                                 (results_df['dropscale'] == comb1[1])]['test_accuracy']
    
            #print(accuracies1)
            
            accuracies2 = results_df[(results_df['dropmode'] == comb2[0]) &
                                 (results_df['dropscale'] == comb2[1])]['test_accuracy']
            
            # Perform Wilcoxon rank-sum test
            stat, p_value = ranksums(accuracies1.values, accuracies2.values)
            
            # Check if the p-value indicates a significant difference
            significant = p_value < 0.05
            
            # Append the result to the significance_results DataFrame
            significance_results.loc[len(significance_results)] = [f"{comb1[0]}_{comb1[1]}", f"{comb2[0]}_{comb2[1]}",  p_value, significant, 
                                                                   np.round(100*np.mean(accuracies1), 2), np.round(100*np.mean(accuracies2), 2), 
                                                                   np.round(100*np.std(accuracies1), 2), np.round(100*np.std(accuracies2), 2)]
           

# Print the significance results
print(significance_results)

significance_results.to_excel(log_folder+file_name+'_sig.xlsx', index=False)

             comb1           comb2   p_value  significant  accuracies1  \
0      dropout_0.0     dropout_1.0  0.028280         True        94.57   
1      dropout_0.0     postact_0.0  0.530869        False        94.57   
2      dropout_0.0     postact_1.0  0.296270        False        94.57   
3      dropout_0.0  postspline_0.0  0.250592        False        94.57   
4      dropout_0.0  postspline_1.0  0.530869        False        94.57   
5      dropout_1.0     postact_0.0  0.009023         True        94.44   
6      dropout_1.0     postact_1.0  0.174525        False        94.44   
7      dropout_1.0  postspline_0.0  0.117185        False        94.44   
8      dropout_1.0  postspline_1.0  0.117185        False        94.44   
9      postact_0.0     postact_1.0  0.028280         True        94.60   
10     postact_0.0  postspline_0.0  0.047202         True        94.60   
11     postact_0.0  postspline_1.0  0.296270        False        94.60   
12     postact_1.0  postspline_0.0  0.

In [168]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision

# Define transformations for the training data and testing data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Download and load the training data
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Download and load the testing data
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Convert datasets to numpy arrays and flatten
train_data = trainset.data.numpy().reshape(-1, 28*28)
train_targets = trainset.targets.numpy()
test_data = testset.data.numpy().reshape(-1, 28*28)
test_targets = testset.targets.numpy()

# Normalize the train and test data to [-1, 1]
train_data = (train_data / 255.0 - 0.5) / 0.5
test_data = (test_data / 255.0 - 0.5) / 0.5

In [169]:
dataset = {}
dataset['train_input'] = torch.from_numpy(train_data).float()
dataset['test_input'] = torch.from_numpy(test_data).float()
dataset['train_label'] = torch.from_numpy(train_targets)
dataset['test_label'] = torch.from_numpy(test_targets)


In [153]:
model = DropKAN(
        seed=j,
        width=[dataset['train_input'].shape[1], 10, 1],
        symbolic_enabled=False,
        drop_rate=[0., 0.],
        drop_mode='postact',
        drop_scale=True,
        neuron_fun=['sum', 'sum']
        )
results = model.train(dataset, opt="Adam", steps=1000, batch=256, lr=0.1, update_grid=False)
set_training_mode(model, False)

#pred = model(dataset['train_input'])
#probabilities = torch.softmax(pred, dim=1)
#predicted_classes = torch.argmax(probabilities, dim=1)
#train_acc = 100*(predicted_classes == dataset['train_label']).sum().item()/dataset['train_label'].size(0)
pred = model(dataset['test_input'])
probabilities = torch.softmax(pred, dim=1)
predicted_classes = torch.argmax(probabilities, dim=1)
test_acc = 100*(predicted_classes == dataset['test_label']).sum().item()/dataset['test_label'].size(0)
print(test_acc)

description: 100%|██████████████████████████████████████████████| 1000/1000 [01:21<00:00, 12.23it/s]


9.8


In [160]:
np.min(train_data)

-0.424212917883804

In [156]:
pred

tensor([[4.5938],
        [4.5726],
        [4.5822],
        ...,
        [4.5987],
        [4.5869],
        [4.5724]], grad_fn=<AddBackward0>)

In [111]:
log_df = pd.DataFrame(columns=['drop_rate', 'drop_mode', 'scale', 'test_accuracy'])

model = DropKAN(
        seed=j,
        width=[dataset['train_input'].shape[1], 10, 10],
        grid=5,
        symbolic_enabled=False,
        drop_rate=[0.0, 0.0],
        drop_mode=None,
        drop_scale=None,
        neuron_fun=['sum', 'sum']
        )
results = model.train(dataset, opt="Adam", steps=10000, batch=32, lr=0.01, update_grid=False, loss_fn=nn.CrossEntropyLoss())
set_training_mode(model, False)

#pred = model(dataset['train_input'])
#probabilities = torch.softmax(pred, dim=1)
#predicted_classes = torch.argmax(probabilities, dim=1)
#train_acc = 100*(predicted_classes == dataset['train_label']).sum().item()/dataset['train_label'].size(0)
pred = model(dataset['test_input'])
probabilities = torch.softmax(pred, dim=1)
predicted_classes = torch.argmax(probabilities, dim=1)
test_acc = 100*(predicted_classes == dataset['test_label']).sum().item()/dataset['test_label'].size(0)
new_row = [None,'kan',None,test_acc]
            
log_df.loc[len(log_df)] = new_row

best = test_acc
print('best:', best)
print()

for drop_rate in np.arange(0.01, 0.2, 0.02):

    for drop_mode in ['postspline', 'postact', 'dropout']:

        for scale in  [True, False]:
            print(drop_rate, drop_mode)
            
            
            
            model = DropKAN(
                    seed=j,
                    width=[dataset['train_input'].shape[1], 10, 10],
                    grid=5,
                    symbolic_enabled=False,
                    drop_rate=[drop_rate, drop_rate],
                    drop_mode=drop_mode,
                    drop_scale=scale,
                    neuron_fun=['sum', 'sum']
                    )
            results = model.train(dataset, opt="Adam", steps=10000, batch=32, lr=0.01, update_grid=False, loss_fn=nn.CrossEntropyLoss())
            set_training_mode(model, False)
            
            #pred = model(dataset['train_input'])
            #probabilities = torch.softmax(pred, dim=1)
            #predicted_classes = torch.argmax(probabilities, dim=1)
            #train_acc = 100*(predicted_classes == dataset['train_label']).sum().item()/dataset['train_label'].size(0)
            pred = model(dataset['test_input'])
            probabilities = torch.softmax(pred, dim=1)
            predicted_classes = torch.argmax(probabilities, dim=1)
            test_acc = 100*(predicted_classes == dataset['test_label']).sum().item()/dataset['test_label'].size(0)
            print(test_acc)

            if test_acc > best:
                best = test_acc
                print('best:', best)
                print()

            new_row = [drop_rate,drop_mode,scale,test_acc]
            
            log_df.loc[len(log_df)] = new_row
        
            # Save the DataFrame to an Excel file
            print(len(log_df))
            print()

description: 100%|████████████████████████████████████████████| 10000/10000 [03:56<00:00, 42.28it/s]


best: 94.24

0.01 postspline


description: 100%|████████████████████████████████████████████| 10000/10000 [04:05<00:00, 40.81it/s]


93.78
2

0.01 postspline


description: 100%|████████████████████████████████████████████| 10000/10000 [04:05<00:00, 40.71it/s]


...

93.08
18

0.049999999999999996 dropout


description: 100%|████████████████████████████████████████████| 10000/10000 [04:00<00:00, 41.65it/s]


93.63
19

0.06999999999999999 postspline


description:  25%|███████████▎                                 | 2508/10000 [00:58<02:55, 42.75it/s]


KeyboardInterrupt: 

In [161]:
log_df.sort_values(by='test_accuracy', ascending=False)

,drop_rate,drop_mode,scale,test_accuracy
15,0.05,postact,True,94.39
6,0.01,dropout,False,94.35
3,0.01,postact,True,94.34
4,0.01,postact,False,94.30
0,NaN,kan,None,94.24
14,0.05,postspline,False,94.24
5,0.01,dropout,True,94.20
2,0.01,postspline,False,94.19
8,0.03,postspline,False,94.19
10,0.03,postact,False,94.13


In [162]:
grouped = log_df.groupby(['drop_mode'])
best_settings = grouped.apply(lambda x: x.loc[x['test_accuracy'].idxmax()])

In [163]:
best_settings

,drop_rate,drop_mode,scale,test_accuracy
drop_mode,,,,
dropout,0.01,dropout,False,94.35
kan,NaN,kan,None,94.24
postact,0.05,postact,True,94.39
postspline,0.05,postspline,False,94.24


In [170]:
grouped = log_df.groupby(['drop_mode'])
best_settings = grouped.apply(lambda x: x.loc[x['test_accuracy'].idxmax()])
results_df = pd.DataFrame(columns=['dropmode', 'dropscale', 'test_accuracy'])

for i, (index, setting) in enumerate(best_settings.iterrows()):
    for j in range(5):
        model = DropKAN(
                    seed=j,
                    width=[dataset['train_input'].shape[1], 10, 10],
                    grid=5,
                    symbolic_enabled=False,
                    drop_rate=[setting['drop_rate'], setting['drop_rate']],
                    drop_mode=setting['drop_mode'],
                    drop_scale=setting['scale'],
                    neuron_fun=['sum', 'sum']
                    )
        results = model.train(dataset, opt="Adam", steps=10000, batch=32, lr=0.01, update_grid=False, loss_fn=nn.CrossEntropyLoss())
        set_training_mode(model, False)
            
            #pred = model(dataset['train_input'])
            #probabilities = torch.softmax(pred, dim=1)
            #predicted_classes = torch.argmax(probabilities, dim=1)
            #train_acc = 100*(predicted_classes == dataset['train_label']).sum().item()/dataset['train_label'].size(0)
        pred = model(dataset['test_input'])
        probabilities = torch.softmax(pred, dim=1)
        predicted_classes = torch.argmax(probabilities, dim=1)
        test_acc = 100*(predicted_classes == dataset['test_label']).sum().item()/dataset['test_label'].size(0)
        print(setting['drop_mode'], setting['scale'], test_acc)
        results_df.loc[len(results_df)] = [setting['drop_mode'], setting['scale'], test_acc]

description: 100%|████████████████████████████████████████████| 10000/10000 [03:58<00:00, 41.99it/s]


dropout False 94.35


description: 100%|████████████████████████████████████████████| 10000/10000 [04:00<00:00, 41.65it/s]


dropout False 93.38


description: 100%|████████████████████████████████████████████| 10000/10000 [03:58<00:00, 41.91it/s]


...

postspline False 94.01


description: 100%|████████████████████████████████████████████| 10000/10000 [04:02<00:00, 41.29it/s]


postspline False 93.87


description: 100%|████████████████████████████████████████████| 10000/10000 [04:01<00:00, 41.34it/s]


postspline False 93.93


In [172]:
results_df

,dropmode,dropscale,test_accuracy
0,dropout,False,94.35
1,dropout,False,93.38
2,dropout,False,93.96
3,dropout,False,93.95
4,dropout,False,94.05
5,kan,None,94.24
6,kan,None,94.07
7,kan,None,94.01
8,kan,None,93.53
9,kan,None,93.95
